In [2]:
# Import 
import cv2
import mediapipe as mp
import numpy as np

mp_dwg = mp.solutions.drawing_utils
mp_pos = mp.solutions.pose

In [3]:
# Follow Pose Landmark Model from https://google.github.io/mediapipe/solutions/pose#pose_landmarks

In [6]:
# Claculation degree function 

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle


In [7]:
# Curl counter variables
counter_l = 0 # Left
cl = 0         # Left
stage_l = None  # Left
counter_R = 0    # Right
stage_R = None   # Right
source = 0
cap =cv2.VideoCapture(source)
with mp_pos.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        # Change Color to RGB
        img =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        
        #Detection
        results = pose.process(img)
        
        #Change Color back to BGR
        img.flags.writeable = True
        img =cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            #prepare information Left Side
            shoulder = [landmarks[mp_pos.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pos.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pos.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pos.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pos.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pos.PoseLandmark.LEFT_WRIST.value].y]
            
            #prepare information Right Side
            shoulder_R = [landmarks[mp_pos.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pos.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_R = [landmarks[mp_pos.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pos.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_R = [landmarks[mp_pos.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pos.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            angle_l = calculate_angle(shoulder, elbow, wrist)
            angle_R = calculate_angle(shoulder_R, elbow_R, wrist_R)
            
            #Visualize angle Left Side
            cv2.putText(img, str(angle_l), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            #Visualize angle Right Side
            cv2.putText(img, str(angle_R), 
                           tuple(np.multiply(elbow_R, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            # counter Function Left
            if angle_l > 160:
                stage_l = "down"
            if angle_l < 30 and stage_l =='down':
                stage_l="up"
                counter_l +=1
                print(counter_l)
                
            # counter Function Right
            if angle_R > 160:
                stage_R = "down"
            if angle_R < 30 and stage_R =='down':
                stage_R="up"
                counter_R +=1
                print(counter_R)
                
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(img, (0,0), (225,73), (245,117,16), -1)
        cv2.rectangle(img, (415,0), (640,73), (245,117,16), -1)
        
        # Rep data Left Side
        cv2.putText(img, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(img, str(counter_l), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        # Rep data Right Side
        cv2.putText(img, 'REPS', (430,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(img, str(counter_R), 
                    (425,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Stage data Left Side
        cv2.putText(img, 'STAGE', (80,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(img, stage_l, 
                    (80,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Stage data Right Side
        cv2.putText(img, 'STAGE', (495,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(img, stage_R, 
                    (495,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        
        
        
        
        #Start render Detection 
        mp_dwg.draw_landmarks(img, results.pose_landmarks, mp_pos.POSE_CONNECTIONS,
                                mp_dwg.DrawingSpec(color=(200,117,66), thickness=2, circle_radius=2), 
                                mp_dwg.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )    
        
        
        cv2.imshow('Media Check', img)
        if cv2.waitKey(10) & 0xFF ==27:
            break   
    cap.release()
    cv2.destroyAllWindows()

1
2
1
3
2
4
3
5


In [8]:
#len land marks for chacking
len(landmarks)
for lndmrk in mp_pos.PoseLandmark:
    print(lndmrk)
landmarks[mp_pos.PoseLandmark.LEFT_SHOULDER.value].visibility
landmarks[mp_pos.PoseLandmark.LEFT_ELBOW.value]
landmarks[mp_pos.PoseLandmark.LEFT_WRIST.value]

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


x: 0.8415263891220093
y: 1.3822075128555298
z: -0.2009221613407135
visibility: 0.012216715142130852